In [1]:
import sympy as sp
import numpy as np
from IPython.display import display, Math

In [2]:
def transX(d):
    return sp.Matrix([[1, 0, 0, d],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
def transZ(a):
    return sp.Matrix([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, a],
                     [0, 0, 0, 1]])

def rotX(theta):
    return sp.Matrix([[1, 0, 0, 0],
                     [0, sp.cos(theta), -sp.sin(theta), 0],
                     [0, sp.sin(theta), sp.cos(theta), 0],
                     [0, 0, 0, 1]])

def rotZ(theta):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                     [sp.sin(theta), sp.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    
def rotMattoRPY(R):
    return sp.Matrix([sp.atan2(R[1,0], R[0,0]), sp.atan2(-R[2,0], sp.sqrt(R[2,1]**2 + R[2,2]**2)), sp.atan2(R[2,1], R[2,2])])

In [3]:
l0, l1, l2 = sp.Symbol('l0'), sp.Symbol('l1'), sp.Symbol('l2')

DH = sp.Matrix([[0, 0, l0, -sp.pi/2],
               [0, -sp.pi/2, 0, -sp.pi/2],
               [0, sp.pi/2, l1, 0],
               [0, 0, l2, 0]])

#type of joints
jointType = sp.Matrix([1, 1, 1, 0])
# where 1 is revolute and 0 is prismatic

# homogeneous transformation matrix
Hne = sp.Matrix([[0, 0, 1, 0],
                [0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 0, 1]])
display(Math(r'\mathbf{DH} = ' + sp.latex(DH)))
display(Math(r'\mathbf{Hne} = ' + sp.latex(Hne)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
Hiden = sp.eye(4)
q = [sp.Symbol('q1'), sp.Symbol('q2'), sp.Symbol('q3'), sp.Symbol('q4')]
for i in range(DH.shape[0]):
    Tx = transX(DH[i, 0])
    Rx = rotX(DH[i, 1])
    Tz = transZ(DH[i, 2])
    Rz = rotZ(DH[i, 3])
    if jointType[i]:
        Hj = rotZ(q[i])
    else:
        Hj = transZ(q[i])
        
    Hiden = Hiden * Tx * Rx * Tz * Rz * Hj
    
H0e = Hiden * Hne
H0e

Matrix([
[-sin(q1)*cos(q2),  sin(q1)*sin(q2)*sin(q3) - cos(q1)*cos(q3), sin(q1)*sin(q2)*cos(q3) + sin(q3)*cos(q1), -l1*sin(q1)*cos(q2) - l2*sin(q1)*cos(q2) - q4*sin(q1)*cos(q2)],
[ cos(q1)*cos(q2), -sin(q1)*cos(q3) - sin(q2)*sin(q3)*cos(q1), sin(q1)*sin(q3) - sin(q2)*cos(q1)*cos(q3),  l1*cos(q1)*cos(q2) + l2*cos(q1)*cos(q2) + q4*cos(q1)*cos(q2)],
[         sin(q2),                            sin(q3)*cos(q2),                           cos(q2)*cos(q3),                     l0 + l1*sin(q2) + l2*sin(q2) + q4*sin(q2)],
[               0,                                          0,                                         0,                                                             1]])

In [5]:
R = sp.trigsimp(H0e[0:3, 0:3])
P = sp.trigsimp(H0e[0:3, 3])

display(Math(r'\mathbf{R} = ' + sp.latex(R)))
display(Math(r'\mathbf{P} = ' + sp.latex(P)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
sp.latex(P)

'\\left[\\begin{matrix}- \\left(l_{1} + l_{2} + q_{4}\\right) \\sin{\\left(q_{1} \\right)} \\cos{\\left(q_{2} \\right)}\\\\\\left(l_{1} + l_{2} + q_{4}\\right) \\cos{\\left(q_{1} \\right)} \\cos{\\left(q_{2} \\right)}\\\\l_{0} + l_{1} \\sin{\\left(q_{2} \\right)} + l_{2} \\sin{\\left(q_{2} \\right)} + q_{4} \\sin{\\left(q_{2} \\right)}\\end{matrix}\\right]'

In [7]:
sp.pycode(R)

'ImmutableDenseMatrix([[-math.sin(q1)*math.cos(q2), math.sin(q1)*math.sin(q2)*math.sin(q3) - math.cos(q1)*math.cos(q3), math.sin(q1)*math.sin(q2)*math.cos(q3) + math.sin(q3)*math.cos(q1)], [math.cos(q1)*math.cos(q2), -math.sin(q1)*math.cos(q3) - math.sin(q2)*math.sin(q3)*math.cos(q1), math.sin(q1)*math.sin(q3) - math.sin(q2)*math.cos(q1)*math.cos(q3)], [math.sin(q2), math.sin(q3)*math.cos(q2), math.cos(q2)*math.cos(q3)]])'

In [8]:
sp.pycode(P)

'ImmutableDenseMatrix([[-(l1 + l2 + q4)*math.sin(q1)*math.cos(q2)], [(l1 + l2 + q4)*math.cos(q1)*math.cos(q2)], [l0 + l1*math.sin(q2) + l2*math.sin(q2) + q4*math.sin(q2)]])'

In [9]:
[[-np.sin(q1)*np.cos(q2), np.sin(q1)*np.sin(q2)*np.sin(q3) - np.cos(q1)*np.cos(q3), np.sin(q1)*np.sin(q2)*np.cos(q3) + np.sin(q3)*np.cos(q1)],
 [np.cos(q1)*np.cos(q2), -np.sin(q1)*np.cos(q3) - np.sin(q2)*np.sin(q3)*np.cos(q1), np.sin(q1)*np.sin(q3) - np.sin(q2)*np.cos(q1)*np.cos(q3)],
 [np.sin(q2), np.sin(q3)*np.cos(q2), np.cos(q2)*np.cos(q3)]]

[[(-l1 - l2 - q4)*np.sin(q1)*np.cos(q2)],
 [(l1 + l2 + q4)*np.cos(q1)*np.cos(q2)],
 [l0 + l1*np.sin(q2) + l2*np.sin(q2) + q4*np.sin(q2)]]

NameError: name 'q1' is not defined

In [ ]:
testArr = np.array([[1],[2],[3]])
testArr[1, 0]

2

In [ ]:
degree = np.pi/180.0
[q1,q2,q3,q4] = [30.0,60.0,90.0,0.5]
[q1, q2, q3] = [i*degree for i in [q1, q2, q3]]

print(q1, q2, q3, q4)

0.5235987755982988 1.0471975511965976 1.5707963267948966 0.5


In [ ]:
# testArr2 = np.array('d', [[1],[2],[3]])

TypeError: data type not understood

In [ ]:
x, y, z = sp.symbols('x y z')
Peq1 = sp.Eq(x, P[0])
Peq2 = sp.Eq(y, P[1])
Peq3 = sp.Eq(z, P[2])
newP = sp.eye(3,1)

newQ = sp.Matrix([sp.Symbol('q1'), sp.Symbol('q2'), sp.Symbol('q4')])

result = sp.solve([Peq1, Peq2, Peq3], (q[0], q[1], q[3]))

In [ ]:
gamma = sp.symbols('gamma')

In [ ]:
Q = sp.Matrix([[-sp.atan(x/y)],
               [sp.asin(sp.sqrt(1-((x**2+y**2)/((gamma*sp.sqrt(((z-l0)**2)+x**2+y**2))**2))))],
               [-l1 - l2 + gamma*(sp.sqrt(((z-l0)**2)+x**2+y**2))]])

Matrix([
[                                                           -atan(x/y)],
[asin(sqrt(1 - (x**2 + y**2)/(gamma**2*(x**2 + y**2 + (-l0 + z)**2))))],
[                     gamma*sqrt(x**2 + y**2 + (-l0 + z)**2) - l1 - l2]])

In [ ]:
([[-np.atan(x/y)],
  [np.asin(np.sqrt(1-((x**2+y**2)/((gamma*np.sqrt(((z-l0)**2)+x**2+y**2))**2))))],
  [-l1 - l2 + gamma*(np.sqrt(((z-l0)**2)+x**2+y**2))]])

In [ ]:
linkLengths = [0.4, 0.15, 0.2]
x , y , z = [0.0, 0.0, 0.4]
[gamma1, gamma2, gamma3] = [1, 1, 1]

In [ ]:
IKEq = np.array([[-np.arctan2(x/gamma1,y/gamma1)],
                 [np.arcsin(gamma2*np.sqrt(1-((x**2+y**2)/((np.sqrt(((z-linkLengths[0])**2)+x**2+y**2))**2))))],
                 [-linkLengths[1] - linkLengths[2] + gamma3*(np.sqrt(((z-linkLengths[0])**2)+x**2+y**2))]])
IKEq

/tmp/ipykernel_3854/101058236.py:2: RuntimeWarning: invalid value encountered in double_scalars
  [np.arcsin(gamma2*np.sqrt(1-((x**2+y**2)/((np.sqrt(((z-linkLengths[0])**2)+x**2+y**2))**2))))],


array([[-0.  ],
       [  nan],
       [-0.35]])

In [ ]:
np.pi

3.141592653589793

In [ ]:
def checkIK(self, gamma, q, z):
    if((gamma[1]) and (gamma[2]) and (q[1] >= 0 and q[1] <= np.pi) and (q[3] >= 0 and q[3] <= 0.1) and (z >= 0.4 and z <= 0.85)):
        return True
    return False

In [ ]:
pos = [-0.071, 0.345, 0.520]
ptInWS = np.sqrt(pos[0]**2 + pos[1]**2 + (pos[2] - 0.4)**2)
ptInWS

0.37211019873150475

In [ ]:
def test():
    x, y, z = 1, 2, 3
    return x, y, z

var1 = test()[0]
var1

1

In [ ]:
x = 0.5
if 0 <= x <= 1:
    print('yes')
else:
    print('no')

yes


In [ ]:
def test():
    return 'oh hi'
    
flag = True
st = test() if flag else 'oh bye'
print(st)

oh hi
